## Heart Disease Classifier
This files classifies whether or not a patient has heart disease based on his medical attributes

#### Problem Definition
> Given medical attributes of a person, predict if he has heart disease or not

#### Data
> The dataset references the UC Irivine ML repository.
https://archive.ics.uci.edu/dataset/45/heart+disease

#### Evaluation
> The objective is to attain a minimum accuracy of 95% and to validate the model via verifiable proof-of-work artifacts, including training logs, evaluation metrics, and reproducible code.

#### Features
* age
* sex
* chest pain type (4 values)
* resting blood pressure
* serum cholestoral in mg/dl
* fasting blood sugar > 120 mg/dl
* resting electrocardiographic results (values 0,1,2)
* maximum heart rate achieved
* exercise induced angina
* oldpeak = ST depression induced by exercise relative to rest
* the slope of the peak exercise ST segment
* number of major vessels (0-3) colored by flourosopy
* thal: 0 = normal; 1 = fixed defect; 2 = reversable defect

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
#models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

#tools
from sklearn.model_selection import train_test_split as tts, cross_val_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.metrics import classification_report, confusion_matrix